In [1]:
import os

path = "/Users/saramcghee/.cache/kagglehub/datasets/veeralakrishna/200-bird-species-with-11788-images/versions/1"
print(os.listdir(path))


['CUB_200_2011.tgz', 'segmentations.tgz']


In [4]:
import os, shutil, random
from pathlib import Path

images_dir = "/Users/saramcghee/.cache/kagglehub/datasets/veeralakrishna/200-bird-species-with-11788-images/versions/1/CUB_200_2011/CUB_200_2011/images"

split_root = os.path.join(os.path.dirname(images_dir), "splits")
train_root = os.path.join(split_root, "train")
valid_root = os.path.join(split_root, "valid")

# Only do this once
if not os.path.exists(split_root):
    os.makedirs(train_root, exist_ok=True)
    os.makedirs(valid_root, exist_ok=True)

    random.seed(42)
    val_ratio = 0.2

    class_dirs = [d for d in os.listdir(images_dir) if os.path.isdir(os.path.join(images_dir, d))]
    print("Classes:", len(class_dirs))

    for cls in class_dirs:
        src_cls = os.path.join(images_dir, cls)
        images = [f for f in os.listdir(src_cls) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
        random.shuffle(images)

        n_val = int(len(images) * val_ratio)
        val_imgs = images[:n_val]
        train_imgs = images[n_val:]

        dst_train_cls = os.path.join(train_root, cls)
        dst_val_cls = os.path.join(valid_root, cls)
        os.makedirs(dst_train_cls, exist_ok=True)
        os.makedirs(dst_val_cls, exist_ok=True)

        for f in train_imgs:
            shutil.copy2(os.path.join(src_cls, f), os.path.join(dst_train_cls, f))
        for f in val_imgs:
            shutil.copy2(os.path.join(src_cls, f), os.path.join(dst_val_cls, f))

    print("✅ Split created at:", split_root)
else:
    print("Split already exists at:", split_root)

print("train_root:", train_root)
print("valid_root:", valid_root)


Classes: 200
✅ Split created at: /Users/saramcghee/.cache/kagglehub/datasets/veeralakrishna/200-bird-species-with-11788-images/versions/1/CUB_200_2011/CUB_200_2011/splits
train_root: /Users/saramcghee/.cache/kagglehub/datasets/veeralakrishna/200-bird-species-with-11788-images/versions/1/CUB_200_2011/CUB_200_2011/splits/train
valid_root: /Users/saramcghee/.cache/kagglehub/datasets/veeralakrishna/200-bird-species-with-11788-images/versions/1/CUB_200_2011/CUB_200_2011/splits/valid


In [9]:
# 🔓 Unfreeze last ResNet block + classifier
for name, param in model.named_parameters():
    if name.startswith("layer4") or name.startswith("fc"):
        param.requires_grad = True
    else:
        param.requires_grad = False

# Smaller learning rate for fine-tuning
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4
)

criterion = nn.CrossEntropyLoss()


In [5]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = "mps" if torch.backends.mps.is_available() else "cpu"
print("Device:", device)

train_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

valid_tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_ds = datasets.ImageFolder(train_root, transform=train_tfms)
valid_ds = datasets.ImageFolder(valid_root, transform=valid_tfms)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
valid_loader = DataLoader(valid_ds, batch_size=32, shuffle=False, num_workers=2)

num_classes = len(train_ds.classes)
print("Classes:", num_classes)
print("Train images:", len(train_ds))
print("Valid images:", len(valid_ds))


Device: mps
Classes: 200
Train images: 9465
Valid images: 2323


In [10]:
# 🔓 Unfreeze last ResNet block + classifier
for name, param in model.named_parameters():
    if name.startswith("layer4") or name.startswith("fc"):
        param.requires_grad = True
    else:
        param.requires_grad = False

# Smaller learning rate for fine-tuning
optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4
)

criterion = nn.CrossEntropyLoss()


In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)

def accuracy(logits, y):
    preds = logits.argmax(dim=1)
    return (preds == y).float().mean().item()

for epoch in range(3):  # keep small for learning
    model.train()
    train_acc = 0
    train_loss = 0

    for X, y in train_loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(X)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        train_acc += accuracy(logits, y)

    model.eval()
    val_acc = 0
    val_loss = 0
    with torch.no_grad():
        for X, y in valid_loader:
            X, y = X.to(device), y.to(device)
            logits = model(X)
            loss = criterion(logits, y)

            val_loss += loss.item()
            val_acc += accuracy(logits, y)

    print(
        f"Epoch {epoch+1}/3 | "
        f"Train loss {train_loss/len(train_loader):.4f} acc {train_acc/len(train_loader):.4f} | "
        f"Valid loss {val_loss/len(valid_loader):.4f} acc {val_acc/len(valid_loader):.4f}"
    )


Epoch 1/3 | Train loss 1.4931 acc 0.6400 | Valid loss 1.8161 acc 0.5336
Epoch 2/3 | Train loss 1.2187 acc 0.7028 | Valid loss 1.7548 acc 0.5333
Epoch 3/3 | Train loss 1.0836 acc 0.7318 | Valid loss 1.7124 acc 0.5519


In [12]:
train_ds.classes[:10]


['001.Black_footed_Albatross',
 '002.Laysan_Albatross',
 '003.Sooty_Albatross',
 '004.Groove_billed_Ani',
 '005.Crested_Auklet',
 '006.Least_Auklet',
 '007.Parakeet_Auklet',
 '008.Rhinoceros_Auklet',
 '009.Brewer_Blackbird',
 '010.Red_winged_Blackbird']

In [13]:
torch.save({
    "model_state": model.state_dict(),
    "classes": train_ds.classes
}, "image_model.pth")

print("✅ Image artifacts saved")


✅ Image artifacts saved
